# Blokkmark

## Sølv-tabell

### Pre-prosessering

Henter skjema og oppretter tabell

In [0]:
%pip install tqdm

In [0]:
import pyproj
pyproj.datadir.set_data_dir("/local_disk0/.ephemeral_nfs/cluster_libraries/python/lib/python3.11/site-packages/pyproj/proj_dir/share/proj")

import os
os.environ["GTIFF_SRS_SOURCE"] = "EPSG"

os.environ["PROJ_LIB"] = "/local_disk0/.ephemeral_nfs/cluster_libraries/python/lib/python3.11/site-packages/pyproj/proj_dir/share/proj"

import geopandas as gpd
import numpy as np
import os
import rasterio
import requests
import shutil
import time

from delta.tables import DeltaTable
from dotenv import load_dotenv
from pathlib import Path
from pyspark.sql import DataFrame
from pyspark.sql.functions import *
from pyspark.sql.types import ArrayType, BinaryType, DoubleType, StringType, StructField, StructType
from rasterio.features import rasterize
from rasterio.transform import from_bounds
from rasterio.windows import bounds, Window
from shapely import wkb
from shapely.geometry import box, shape
from shapely.ops import unary_union
from shapely.wkb import dumps
from tqdm import tqdm

In [0]:
catalog_dev = "`land_auto-gen-kart_dev`"
schema_dev = "dl_bildesegmentering"
spark.sql(f"USE CATALOG {catalog_dev}")
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {schema_dev}")
spark.sql(f"USE SCHEMA {schema_dev}")
bronze_table = "blokkmark_bronze"
silver_table = "blokkmark_silver"

BASE_PATH = "/Volumes/land_auto-gen-kart_dev/external_dev/static_data/DL_bildesegmentering"
SUBDIR = {"raster": "blokkmark_raster", "image": "blokkmark_images", "mask": "blokkmark_labels"}

In [0]:
q = f"""
CREATE TABLE IF NOT EXISTS {silver_table} (
    row_hash STRING,
    lokalids ARRAY<STRING>,
    geometry BINARY,
    bbox ARRAY<DOUBLE>,
    bbox_str STRING,
    image_path STRING,
    mask_path STRING,
    image_wms STRING,
    image_status STRING,
    mask_status STRING,
    ingest_time TIMESTAMP,
    photo_time TIMESTAMP
) USING DELTA
"""
spark.sql(q)

### Funksjoner

In [0]:
def read_geometry_from_table() -> DataFrame:
    """
    Henter geometrien til blokkmark og samler det i en data frame.

    Returns:
        DataFrame: Geometrien til blokkmark
    """
    df = (
        spark.read.table(bronze_table)
    )
    return df

def read_nodata_from_table() -> DataFrame:
    """
    Henter geometri-objekter i arealressursflate der det ikke er registrert relevant data og samler det i en data frame.

    Returns:
        DataFrame: Geometrien til arealressursflate
    """
    df = (
        spark.read.table("land_ngis.silver_fkbar5.arealressursflate")
        .select("geometry")
        .filter((col("grunnforhold").isin(98, 99)) & (col("arealtype") == 99))
    )
    return df

In [0]:
def fetch_municipality_boundary(municipality: str) -> gpd.GeoDataFrame:
    """
    Henter polygondata for valgt kommune og returnerer bounding box som en data frame.

    Args:
        municipality (str): Navnet på kommunen

    Returns:
        gpd.GeoDataFrame: Polygondata for kommunen
    """
    EPSG = 4258
    URL_municipality = "https://ws.geonorge.no/kommuneinfo/v1/kommuner?filtrer=kommunenummer,kommunenavn"

    resp = requests.get(URL_municipality)
    resp.raise_for_status()
    municipalities = resp.json()

    municipality_num = None
    for m in municipalities:
        if m["kommunenavn"].lower() == municipality.lower():
            municipality_num = m["kommunenummer"]

    URL_specific = f"https://api.kartverket.no/kommuneinfo/v1/kommuner/{municipality_num}/omrade?utkoordsys={EPSG}&filtrer=omrade"

    resp2 = requests.get(URL_specific)
    resp2.raise_for_status()
    data = resp2.json()

    geom = shape(data["omrade"])
    minx, miny, maxx, maxy = geom.bounds
    bbox = box(minx, miny, maxx, maxy)

    gdf = gpd.GeoDataFrame(
        {"kommunenummer": [municipality_num], "kommunenavn": [municipality]},
        geometry=[bbox],
        crs=f"EPSG:{EPSG}"
    )
    
    return gdf

In [0]:
def create_GeoDataFrame(area: gpd.GeoDataFrame, nodata: bool=False) -> gpd.GeoDataFrame:
    """
    Leser polygondata fra database og returnerer den dataen som overlapper området avgrenset av area.

    Args:
        area (gpd.GeoDataFrame): Bounding box av kommunen i data frame

    Returns:
        gpd.GeoDataFrame: GeoDataFrame med polygondata for kommunen
    """
    if nodata:
        df = read_nodata_from_table()
    else:
        df = read_geometry_from_table()

    df = df.withColumn("ingest_time", current_timestamp())
    
    pdf = df.toPandas()
    pdf["geometry"] = pdf["geometry"].apply(lambda x: wkb.loads(x))
    
    gdf = gpd.GeoDataFrame(pdf, geometry="geometry", crs="EPSG:4258")

    gdf = gpd.clip(gdf, area)
    
    return gdf

In [0]:
def create_raster(gdf: gpd.GeoDataFrame, boundary: gpd.GeoDataFrame, local_path: str, path: str, multi: bool=True) -> None:
    """
    Lager raster av input dataen slik at der det er data blir rasteren hvit, og ellers sort.

    Args:
        gdf (gpd.GeoDataFrame): Polygondata for kommunen
        boundary (gpd.GeoDataFrame): Bounding box av kommunen i data frame
        local_path (str): Sti til raster-fil i midlertidig plassering
        path (str): Sti til raster-fil i endelig polassering
        multi (bool, optional): Om rasteren skal være binær eller farget, default er True
    """
    pixel_size = 1.0

    minx, miny, maxx, maxy = boundary.total_bounds
    width = int(np.ceil((maxx - minx) / pixel_size))
    height = int(np.ceil((maxy - miny) / pixel_size))
    
    transform = from_bounds(minx, miny, maxx, maxy, width, height)

    if multi:
        create_coloured_raster(gdf, local_path, height, width, transform)
    else:
        create_binary_raster(gdf, local_path, height, width, transform)
    
    shutil.copyfile(local_path, path)

    os.remove(local_path)

def create_binary_raster(gdf: gpd.GeoDataFrame, local_path: str, height: int, width: int, transform: rasterio.transform.Affine) -> None:
    """
    Lager binær raster av input dataen.
    """
    shapes = ((row.geometry, 255) for _, row in gdf.iterrows())

    raster = rasterize(
        shapes,
        out_shape=(height, width),
        transform=transform,
        fill=0,
        dtype="uint8"
    )

    with rasterio.open(
        local_path, 'w',
        driver='GTiff',
        height=height,
        width=width,
        count=1,
        dtype='uint8',
        crs=gdf.crs,
        transform=transform,
        nodata=0,
        compress='lzw'
    ) as dst:
        dst.write(raster, 1)

def create_coloured_raster(gdf: gpd.GeoDataFrame, local_path: str, height: int, width: int, transform: rasterio.transform.Affine) -> None:
    """
    Lager fargelagt raster av input dataen.
    """
    colour_mapping = {
        30: (0, 255, 0, 255),     # grønn
        50: (255, 255, 0, 255)    # gul
    }
    
    shapes_r = ((row.geometry, colour_mapping[int(row["arealtype"])][0]) for _, row in gdf.iterrows())
    shapes_g = ((row.geometry, colour_mapping[int(row["arealtype"])][1]) for _, row in gdf.iterrows())
    shapes_b = ((row.geometry, colour_mapping[int(row["arealtype"])][2]) for _, row in gdf.iterrows())

    raster_r = rasterize(shapes_r, out_shape=(height, width), transform=transform, fill=0, dtype="uint8")
    raster_g = rasterize(shapes_g, out_shape=(height, width), transform=transform, fill=0, dtype="uint8")
    raster_b = rasterize(shapes_b, out_shape=(height, width), transform=transform, fill=0, dtype="uint8")

    with rasterio.open(
        local_path, 'w',
        driver='GTiff',
        height=height,
        width=width,
        count=3,
        dtype="uint8",
        crs=gdf.crs,
        transform=transform,
        nodata=0,
        compress='lzw'
    ) as dst:
        dst.write(raster_r, 1)  # R
        dst.write(raster_g, 2)  # G
        dst.write(raster_b, 3)  # B

In [0]:
def pre_process_image_data(municipality: str, CRS: int, local_raster: str, path_nodata: str, path_blokkmark: str) -> None:
    """
    Forbereder generering av treningsdata i gyldige områder ved å lage raster av relevant data og ugyldige områder avgrenset av ønsket område.

    Args:
        municipality (str): Kommunenavn som skal avgrense området
        CRS (int): Koordinatsystem som skal brukes
        local_raster (str) Filsti til lokal midlertidig raster
        path_nodata (str): Filsti til endelig raster over ugyldige områder
        path_blokkmark (str): Filsti til endelig raster over relevant data
    """
    municipality_gdf = fetch_municipality_boundary(municipality=municipality)
    municipality_projected_gdf = municipality_gdf.to_crs(f"EPSG:{CRS}")

    # ========================================
    # Prosesserer ugyldig data
    # ========================================
    gdf_all = create_GeoDataFrame(area=municipality_gdf, nodata=True)
    gdf_all = gdf_all.to_crs(f"EPSG:{CRS}")
    create_raster(gdf=gdf_all, boundary=municipality_projected_gdf, local_path=local_raster, path=path_nodata, multi=False)
    
    # ========================================
    # Prosesserer blokkmark
    # ========================================

    gdf = create_GeoDataFrame(area=municipality_gdf)
    gdf = gdf.to_crs(f"EPSG:{CRS}")
    create_raster(gdf=gdf, boundary=municipality_projected_gdf, local_path=local_raster, path=path_blokkmark)

In [0]:
def generate_bboxes(src_path: str, out_of_bounds: str) -> tuple:
    """
    Genererer bounding box, både på array- og streng-format for alle tiles.

    Args:
        src_path (str): Filsti til raster-fil over blokkmark-områder
        out_of_bounds (str): Filsti til raster-fil over ugyldige områder

    Returns:
        tuple:
            - bbox_arrays (list): Liste med bounding box på array-format
            - bbox_strings (list): Liste med bounding box på streng-format
    """
    tile_size = 1024

    bbox_arrays = []
    bbox_strings = []

    with rasterio.open(src_path) as src, rasterio.open(out_of_bounds) as oob:
        width = src.width
        height = src.height
        crs = src.crs

        oob_data = oob.read(1)

        n_cols = int(np.ceil(width / tile_size))
        n_rows = int(np.ceil(height / tile_size))

        for r in tqdm(range(n_rows), desc="🧩 Bygger tile grenser & stier 🛤️", colour="yellow"):
            for c in range(n_cols):
                x_off = c * tile_size
                y_off = r * tile_size

                if x_off + tile_size > width or y_off + tile_size > height:
                    continue

                tile = oob_data[y_off:y_off+tile_size, x_off:x_off+tile_size]

                if np.any(tile == 255):
                    continue

                window = Window(x_off, y_off, tile_size, tile_size)
                left, bottom, right, top = bounds(window, src.transform)

                bbox_array = [left, bottom, right, top]
                bbox_str = f"{left},{bottom},{right},{top}"

                bbox_arrays.append(bbox_array)
                bbox_strings.append(bbox_str)
    
    return bbox_arrays, bbox_strings

In [0]:
def generate_multiPolygons_and_IDs(df: DataFrame, bbox_arrays: list, CRS: int) -> tuple:
    """
    For hver tile-fil genereres det et multipolygon med geometriene inni bounding box i tillegg til en liste av ID‑ene til disse geometriene i bronse-tabellen.

    Args:
        df (DataFrame): Bronse-tabell med geometrier
        bbox_arrays (list): Liste med bounding box på array-format
        CRS (int): Koordinatsystem som skal brukes

    Returns:
        tuple:
            - multipolygons (list): Liste med multipolygons
            - IDs (list): Liste med lister av ID‑ene til geometriene inne i bounding box
    """
    bbox_df = spark.createDataFrame(
        [(i, box(*bbox).wkt) for i, bbox in enumerate(bbox_arrays)],
        ["bbox_id", "bbox_wkt"]
    ).withColumn("bbox_geom", expr(f"ST_GeomFromText(bbox_wkt, {CRS})"))

    joined = bbox_df.alias("b").join(
        broadcast(df).alias("a"),
        expr("ST_Intersects(a.geometry, b.bbox_geom)"),
        how="left"
    )

    clipped = joined.selectExpr(
        "b.bbox_id as bbox_id",
        "ST_Intersection(a.geometry, b.bbox_geom) as geom",
        "a.lokalid as lokalid"
    )
    
    ids_per_bbox = clipped.groupBy("bbox_id").agg(
        collect_list("lokalid").alias("ids")
    )

    multipoly_per_bbox = clipped.groupBy("bbox_id").agg(
        expr("ST_Union(collect_list(geom))").alias("multipoly")
    )

    final_df = bbox_df.select("bbox_id").join(ids_per_bbox, ["bbox_id"], "left") \
                      .join(multipoly_per_bbox, ["bbox_id"], "left")
    
    final_df = final_df.select(
        "bbox_id",
        expr("CASE WHEN ids IS NOT NULL THEN ids ELSE array() END").alias("ids"),
        "multipoly"
    ).orderBy("bbox_id")

    rows = final_df.select("multipoly", "ids").collect()
    multipolygons = [r["multipoly"] if r["multipoly"] else None for r in rows]
    IDs = [r["ids"] if r["ids"] is not None else [] for r in rows]

    return multipolygons, IDs

In [0]:
def generate_image_url(bbox_str: str) -> str:
    """
    Genererer en URL for image-bilde basert på bbox_str.
    """
    width, height = [1024, 1024]
    return (
        f"https://wms.geonorge.no/skwms1/wms.nib?VERSION=1.3.0"
        f"&service=WMS&request=GetMap&Format=image/png&"
        f"GetFeatureInfo=text/plain&CRS=EPSG:25833&Layers=ortofoto&"
        f"BBox={bbox_str}&width={width}&height={height}"
    )

def image_file_exists(id: str) -> str:
    """
    Sjekker om bildet med gitt ID er lastet ned.
    """
    path = f"/Volumes/land_auto-gen-kart_dev/external_dev/static_data/DL_bildesegmentering/helipad_images/image_{id}.png"
    return "DOWNLOADED" if os.path.exists(path) else "PENDING"


def mask_file_exists(id: str) -> str:
    """
    Sjekker om masken med gitt ID er generert.
    """
    path = f"/Volumes/land_auto-gen-kart_dev/external_dev/static_data/DL_bildesegmentering/helipad_labels/mask_{id}.png"
    return "GENERATED" if os.path.exists(path) else "PENDING"

In [0]:
load_dotenv()
BRUKERID = os.getenv("GEONORGE_BRUKERID")
PASSORD = os.getenv("GEONORGE_PASSORD")


def get_token():
    """
    Henter token fra GeoNorge og returnerer det.
    """
    url = (
        f"https://baat.geonorge.no/skbaatts/req?brukerid={BRUKERID}"
        f"&passord={PASSORD}&tjenesteid=wms.nib&retformat=s"
    )
    raw_token = requests.get(url).text.strip("`")
    return raw_token


token = get_token()
token_start_time = time.time()
token_lifetime = 55 * 60


def refresh_token_if_needed():
    """
    Henter ny token om den gamle er utløpt.
    """
    global token, token_start_time
    if time.time() - token_start_time > token_lifetime:
        print("🔄 Fornyer token...")
        token = get_token()
        token_start_time = time.time()

In [0]:
def enrich_output(df: DataFrame, token: str) -> DataFrame:
    """
    Legger til kolonner med info om bilder, masker og tid.
    """
    for dt in ["image", "mask"]:
        sub = SUBDIR[dt]
        df = df.withColumn(
            f"{dt}_path",
            concat(
                lit(f"{BASE_PATH}/{sub}/{dt}_"), col("row_hash"), lit(".png")
            ),
        )

    generate_image_url_udf = udf(generate_image_url, StringType())
    image_file_exists_udf = udf(image_file_exists, StringType())
    mask_file_exists_udf = udf(mask_file_exists, StringType())

    df = (
        df.withColumn("image_wms", generate_image_url_udf("bbox_str"))
        .withColumn("image_status", image_file_exists_udf("row_hash"))
        .withColumn("mask_status", mask_file_exists_udf("row_hash"))
        .withColumn("ingest_time", current_timestamp())
    )
    return df

In [0]:
def write_delta_table(sdf: DataFrame, mode: str = "merge") -> None:
    """
    Skriver data til deltatabellen og opdaterer dersom row_hash allerede finnes.
    """
    if mode == "overwrite":
        sdf.write.format("delta").option("mergeSchema", "true").mode(
            "overwrite"
        ).saveAsTable(silver_table)
    else:
        delta_tbl = DeltaTable.forName(spark, silver_table)

        delta_tbl.alias("target").merge(
            sdf.alias("source"), condition="target.row_hash = source.row_hash"
        ).whenMatchedUpdate(
            condition="target.ingest_time < source.ingest_time OR target.image_path IS NULL",
            set={col: f"source.{col}" for col in sdf.columns},
        ).whenNotMatchedInsert(
            values={col: f"source.{col}" for col in sdf.columns}
        ).execute()

### Program

In [0]:
def main() -> None:
    raster_dir = BASE_PATH + "/" + SUBDIR["raster"]
    Path(raster_dir).mkdir(parents=True, exist_ok=True)
    local_raster = r"local_raster.tif"
    path_blokkmark = raster_dir + r"/Blokkmark.tif"
    path_nodata = raster_dir + r"/Out_of_bounds.tif"

    municipality = r"Orkland"
    CRS = 25833

    print("🚀 Starter generering av sølv-tabell ✨")

    # --- Preprocess (kun ved behov) ---
    """
    print("🛠️ Starter preprocess av raster-data ⚙️")
    pre_process_image_data(
        municipality=municipality,
        CRS=CRS,
        local_raster=local_raster,
        path_nodata=path_nodata,
        path_blokkmark=path_blokkmark
    )
    print("🗺️ Raster-filer generert ✅")
    #"""

    # --- Les bronse-geometrier ---
    print("📥 Leser bronse-geometrier fra tabell 🪙")
    df_bronze = read_geometry_from_table()
    df_bronze = (
        df_bronze.withColumn("geometry", expr("ST_GeomFromWKB(geometry)"))
        .withColumn("_geom_SRID", expr("ST_SetSRID(geometry, 4258)"))
        .withColumn("geometry", expr(f"ST_Force2D(ST_Transform(_geom_SRID, 'EPSG:{CRS}'))"))
    )
    print("📐 Bronse-geometrier transformert til riktig SRID 🔄")

    # --- Generer tiles ---
    print("🧩 Genererer tiles og bounding boxes 📦")
    bbox_arrays, bbox_strings = generate_bboxes(
        src_path=path_blokkmark,
        out_of_bounds=path_nodata
    )
    print("🗂️ Tiles og bounding boxes generert 🎯")

    # --- Beregn multipolygoner og IDs ---
    print("🔎 Genererer multipolygoner og ID-lister 📋")
    multipolygons, IDs = generate_multiPolygons_and_IDs(
        df=df_bronze,
        bbox_arrays=bbox_arrays,
        CRS=CRS
    )
    print("🌀 Multipolygoner generert 🗺️")

    # --- Bygg rader ---
    print("🛠️ Bygger rader 📋")
    tile_rows = []
    for ids, mp, arr, bstr in zip(
        IDs,
        multipolygons,
        bbox_arrays,
        bbox_strings
    ):
        mp_wkb = dumps(mp) if mp else None
        tile_rows.append((
            ids,
            mp_wkb,
            arr,
            bstr
        ))

    schema = StructType([
        StructField("lokalids", ArrayType(StringType()), False),
        StructField("geometry", BinaryType(), True),
        StructField("bbox", ArrayType(DoubleType()), False),
        StructField("bbox_str", StringType(), False)
    ])

    tiles_df = spark.createDataFrame(tile_rows, schema=schema)

    tiles_df = tiles_df.withColumn(
        "row_hash",
        sha2(concat_ws("||", "bbox_str"), 256)
    )

    tiles_df = enrich_output(tiles_df, token)

    print("📦 Rader bygget ✅")

    # --- Skriv til Delta ---
    print("💾 Skriver til sølv-tabell 🪙")
    write_delta_table(tiles_df, mode="merge")
    print("🏁 Sølv-tabell skrevet 🎉")

In [0]:
if __name__ == "__main__":
    main()